In [4]:
from textblob import TextBlob
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import spacy
from nltk.sentiment.vader import SentimentIntensityAnalyzer as NLTKSIA
from transformers import pipeline
from flair.models import TextClassifier
from flair.data import Sentence
import nltk
nltk.download('vader_lexicon')

# Function for TextBlob sentiment analysis
def textblob_sentiment(text):
    analysis = TextBlob(text)
    return analysis.sentiment

# Function for VADER sentiment analysis
def vader_sentiment(text):
    analyzer = SentimentIntensityAnalyzer()
    return analyzer.polarity_scores(text)

# # Function for SpaCy sentiment analysis (Note: requires a custom model or external integration)
# def spacy_sentiment(text):
#     nlp = spacy.load('en_core_web_sm')
#     doc = nlp(text)
#     return [(token.text, token.sentiment) for token in doc]

# Function for NLTK VADER sentiment analysis
def nltk_vader_sentiment(text):
    analyzer = NLTKSIA()
    return analyzer.polarity_scores(text)

# Function for Hugging Face transformers sentiment analysis
def transformers_sentiment(text):
    sentiment_analyzer = pipeline("sentiment-analysis")
    return sentiment_analyzer(text)

# Function for Flair sentiment analysis
def flair_sentiment(text):
    classifier = TextClassifier.load('en-sentiment')
    sentence = Sentence(text)
    classifier.predict(sentence)
    return sentence.labels

# Sample text for testing
text = "I love coding, but sometimes it can be frustrating!"

# Testing each function
print("TextBlob Sentiment:", textblob_sentiment(text))
print("VADER Sentiment:", vader_sentiment(text))
print("SpaCy Sentiment:", spacy_sentiment(text))  # SpaCy doesn't have sentiment analysis by default
print("NLTK VADER Sentiment:", nltk_vader_sentiment(text))
print("Transformers Sentiment:", transformers_sentiment(text))
print("Flair Sentiment:", flair_sentiment(text))


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/renju/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


TextBlob Sentiment: Sentiment(polarity=0.0, subjectivity=0.75)
VADER Sentiment: {'neg': 0.301, 'neu': 0.509, 'pos': 0.189, 'compound': -0.3699}
SpaCy Sentiment: [('I', 0.0), ('love', 0.0), ('coding', 0.0), (',', 0.0), ('but', 0.0), ('sometimes', 0.0), ('it', 0.0), ('can', 0.0), ('be', 0.0), ('frustrating', 0.0), ('!', 0.0)]
NLTK VADER Sentiment: {'neg': 0.325, 'neu': 0.471, 'pos': 0.204, 'compound': -0.3699}
Transformers Sentiment: [{'label': 'NEGATIVE', 'score': 0.9928966164588928}]
Flair Sentiment: ['Sentence[11]: "I love coding, but sometimes it can be frustrating!"'/'POSITIVE' (0.9764)]
